In [ ]:
import requests as rq
import pandas as pd

from bs4 import BeautifulSoup

In [ ]:
all_reviews_dict = {}

In [ ]:
url = 'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=&preliminary=on&spoiler=on&p=1'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}
page = rq.get(url, headers=headers)
print(page.status_code)
soup = BeautifulSoup(page.text, 'html.parser')

200


In [ ]:
soup.find_all(attrs={"class": "review-element js-review-element"})

In [ ]:
url_nopage = 'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=&preliminary=on&spoiler=on&p='
page_num = 1
url = url_nopage + str(page_num)
page = rq.get(url, headers=headers)
review_dict = {}

while page.status_code == 200:
    print(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    for item in soup.find_all(attrs={"class": "review-element js-review-element"}):
        try:
            tag = item(attrs={"class": "tags"})[0].get_text()
            review = item(attrs={"class": "text"})[0].get_text()
            rating = item(attrs={"class": "num"})[0].get_text()
            review_dict[review] = (tag, rating)
        except:
            continue
    page_num +=1
    url = url_nopage + str(page_num)
    page = rq.get(url, headers=headers)

In [27]:
all_reviews_dict.update(review_dict)

In [28]:
len(list(all_reviews_dict.items()))

241894

In [30]:
import pickle

with open('all_reviews_dict.pkl', 'wb') as f:
    pickle.dump(all_reviews_dict, f)

In [2]:
import pickle

with open('all_reviews_dict.pkl', 'rb') as f:
    all_reviews = pickle.load(f)

In [6]:
df = pd.DataFrame(list(all_reviews.values()), columns=['tag', 'rating'], index = list(all_reviews.keys()))
df.reset_index(inplace=True)
df.rename(columns={'index': 'review'}, inplace=True)
df

,review,tag,rating
0,\n\n\n\n\n\n Full Metal Alchemi...,Recommended,9
1,\n\n\n\n\n\n Sousou no Frieren ...,RecommendedPreliminary\n \n ...,10
2,\n\n\n\n\n\n This show is about...,Mixed Feelings,7
3,\n\n\n\n\n\n bUt hE MAde GraDua...,RecommendedSpoiler,9
4,\n\n\n\n\n\n I knew going into ...,Recommended,8
...,...,...,...
241889,\n\n\n\n\n\n A comedy/romance b...,Recommended,7
241890,\n\n\n\n\n\n Saiyuki is one of ...,Recommended,9
241891,"\n\n\n\n\n\n First, let me say ...",Recommended,10
241892,\n\n\n\n\n\n Utawarerumono mana...,Recommended,8


In [7]:
df.to_csv('reviews.csv', sep=';', encoding='utf-8')